### Primeiro Passo

Acessando o arquivo ocorrencia_2010_2020.csv no Libre Office e excluindo as colunas que não serão necessárias

# Etapa de extração

### Importando a biblioteca Pandas

In [1]:
import pandas as pd
import pandera as pa #validacao de dataframe

### Carregamento do arquivo csv

In [2]:
df = pd.read_csv("ocorrencia_2010_2020.csv", sep=";", parse_dates=["ocorrencia_dia"], dayfirst=True)
df.head(3) #exibindo os 3 primeiros

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0


In [3]:
df.tail(3) #exibindo os 3 ultimos

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0
5751,79844,79844,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2020-12-31,13:24:00,0


### Montando o esquema de validação

In [20]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int, required=False),  #coluna que pode vir ou nao vir
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        #checa um tamanho máximo e minimo
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        # é preciso checar se as horas estão entre 0 e 24hrs. Feito com expressões regulares
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r"^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$"),nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [21]:
schema.validate(df) #se exibir o dataframe, então passou pela validação

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


### Visualizando os tipos de dados das colunas carregadas

In [6]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [7]:
#Observamos que ocorrencia_dia é uma data, mas está como object (string)
#Precisamos convertê-lo para data
#Na leitura do csv, passa o parametro parse_dates

In [8]:
#Obtendo o mês de cada data
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5747    12
5748    12
5749    12
5750    12
5751    12
Name: ocorrencia_dia, Length: 5752, dtype: int64

### -> O Dataframe está carregado e podemos começar a validar os dados

## Etapa de Validação

### Vamos checar o conjunto de dados em relação ao tipo de dados estipulados para a leitura desses dados

### Atenção a coluna ocorrencia_dia
Está no formato aaaa/dd/mm . Ao carregar o csv, colocar o parametro dayfirst=True


### Quando não existir data, vem com o dado Nat. Se fosse um texto, viria NaN

### Validação dos tipos de dados

In [9]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

### Mesmo usando o parse_dates, podem haver problemas

#### Se vier alguma data fora do padrão, o parse de datas não é feito

In [10]:
#### Usamos uma biblioteca para montar um esquema de validação do dataframe
#### No caso o esquema vai validar se a coluna ocorrencia_dia é do tipo data
#### Se não for, gera uma excessão
#### instalar pandera

### Montando o esquema de validação do dataframe

#### Monta o esquema logo abaixo do dataframe

# Limpeza de Dados

## O que iremos limpar nos dados
Observando as colunas no Libre, observamos que existem dados que não condizem, como siglas que significam que tal dado nao foi informado.

Quem detem o dominio do projeto vai informar quais os dados que devem ser classificados como dados não informados